In [2]:
using HTTP, JSON, JuliaDB, DataFrames

In [3]:
response = HTTP.get("https://hub.mph.in.gov/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22911880b1-d0bb-4168-a5c1-a52dcc291445%22%20WHERE%20%22PERSONTYPEDESCR%22%20IN%20(%27Pedal%20Cyclist%27,%27Pedestrian%27)%20AND%20%22AGE_GRP%22%20IN%20(%271-4%20Years%27,%275-14%20Years%27,%2715-24%20Years%27)%20AND%20%22COUNTYDESCR%22%20IN%20(%27Marion%27,%27Boone%27,%27Hamilton%27,%27Hancock%27,%27Shelby%27,%27Johnson%27,%27Morgan%27,%27Hendricks%27)")

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Server: nginx/1.10.3 (Ubuntu)
Date: Thu, 29 Aug 2019 21:31:59 GMT
Content-Type: application/json;charset=utf-8
Content-Length: 921011
Connection: keep-alive
Pragma: no-cache
Cache-Control: no-cache

{"help": "https://hub.mph.in.gov/api/3/action/help_show?name=datastore_search_sql", "success": true, "result": {"records": [{"COLLISION_TIME": "07:10", "VEHUSECDE": null, "GENDERCDE": "M", "TRAFFICCNTLOPIND": "Y", "SAFETYEQUUSEDDESCR": "", "SCHOOLZONEIND": "N", "VEHLICSTATECDE": "", "TIMENOTIFIEDAMPMTXT": "PM", "PRIMARYFACTORDESCR": "PEDESTRIAN ACTION", "STATEPROPIND": "", "INVESTCOMPLETEIND": "Y", "RDWYRAMPTXT": "", "RESULTALCHTXT": "", "SPEEDLIMITTXT": "", "MANNERCOLLCDE": "12", "PRECOLLACTCDE": null, "RUMBLESTRIPIND": "N", "LIGHTCONDCDE": "3", "DISTRICT_NUM": "3", "TIMEARRIVEDTXT": "07:21", "EMGERENCY_RUN": "N", "MANNERCOLLDESCR": "Other (Explain in Narrative)", "INJNATURECDE": "12", "INCORPLIMITIND": "Y", "LONGDECIMALNMB": "0", "CONSTRUCTIND":

In [4]:
#data = JSON.parse(String(response.body))
data = response.body |> String |> JSON.parse

Dict{String,Any} with 3 entries:
  "success" => true
  "help"    => "https://hub.mph.in.gov/api/3/action/help_show?name=datastore_se…
  "result"  => Dict{String,Any}("records"=>Any[Dict{String,Any}("TRAVDIRCDE"=>"…

In [ ]:
tab = vcat(DataFrame.(data["result"]["records"])...) |> table